In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [25]:
print(" Installing...\n")
!pip install -q torch transformers
!pip install -q llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu118
!pip install -q diffusers accelerate  
!pip install -q sentence-transformers cloudinary
!pip install -q pandas numpy plotly Pillow requests
print("\n Done!")

 Installing...


 Done!


In [3]:
pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.8/736.8 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 26.0rc2
    Uninstalling packaging-26.0rc2:
      Successfully uninstalled packaging-26.0rc2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.47.0 which is incompatible.
google-colab 1.0.0 requires jupyter-server==2.14.0, but you have jupyter-server 2.12.5 which is incompatible.
google-colab 1.0

In [4]:
import os, torch, requests, pandas as pd, numpy as np, time, json, random
from dataclasses import dataclass, field
from typing import List, Dict, Optional
from datetime import datetime
from PIL import Image
from io import BytesIO
import cloudinary
import cloudinary.uploader
from llama_cpp import Llama
from diffusers import StableDiffusionXLPipeline
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

import logging
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

2026-02-17 20:32:53.112827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771360373.492675      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771360373.588922      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771360374.396113      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771360374.396145      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771360374.396148      55 computation_placer.cc:177] computation placer alr

In [12]:
@dataclass
class Config:
    # Instagram tokens
    PAGE_ACCESS_TOKEN: str = os.getenv("PAGE_ACCESS_TOKEN", "your_page_token_here")

    FACEBOOK_PAGE_ID: str = os.getenv("FACEBOOK_PAGE_ID", "your_page_id")
    INSTAGRAM_USER_ID: str = os.getenv("INSTAGRAM_USER_ID", "your_user_id")
    
    # Cloudinary
    CLOUDINARY_CLOUD_NAME: str = os.getenv("CLOUDINARY_CLOUD_NAME", "your_cloud_name")
    CLOUDINARY_API_KEY: str = os.getenv("CLOUDINARY_API_KEY", "your_api_key")
    CLOUDINARY_API_SECRET: str = os.getenv("CLOUDINARY_API_SECRET", "your_api_secret")
    
    # ═══ PINECONE ═══
    PINECONE_API_KEY: str = os.getenv("PINECONE_API_KEY", "your_pinecone_key")
    PINECONE_INDEX: str = os.getenv("PINECONE_INDEX", "autoagent")
    
    # SDXL settings
    SDXL_MODEL: str = "stabilityai/stable-diffusion-xl-base-1.0"
    IMAGE_SIZE: int = 1024
    NATIVE_RES: int = 1024
    INFERENCE_STEPS: int = 30
    GUIDANCE_SCALE: float = 7.5
    IMAGE_DIR: str = "/kaggle/working/instagram_images"

    
    # LLaMA
    LLAMA_MODEL_PATH: str = "/kaggle/input/model-llama/llama-2-7b-chat.Q4_K_M.gguf"
    
    # Brand
    BRAND_NAME: str = "ShopSmart"
    BRAND_HASHTAGS: List[str] = field(default_factory=lambda: ["#ShopSmart", "#SmartShopping"])
    
    # Budget
    MONTHLY_POST_GOAL: int = 60
    MONTHLY_BUDGET: float = 500.00
    COST_PER_POST: float = 5.00
    
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"

config = Config()
os.makedirs(config.IMAGE_DIR, exist_ok=True)

print(f"Device: {config.DEVICE}")
print(f"Model: {config.SDXL_MODEL}")
print(f"Resolution: {config.IMAGE_SIZE}x{config.IMAGE_SIZE}")


Device: cuda
Model: stabilityai/stable-diffusion-xl-base-1.0
Resolution: 1024x1024


In [7]:
print(" Loading products...")
r = requests.get("https://fakestoreapi.com/products", timeout=15)
products_df = pd.DataFrame(r.json())
print(f" {len(products_df)} products")

 Loading products...
 20 products


In [9]:
class KnowledgeBase:
    def __init__(self, config):
        self.config = config
        try:
            print("\n  Pinecone...")
            self.encoder = SentenceTransformer('all-MiniLM-L6-v2', device=config.DEVICE)
            pc = Pinecone(api_key=config.PINECONE_API_KEY)
            if config.PINECONE_INDEX not in pc.list_indexes().names():
                pc.create_index(name=config.PINECONE_INDEX, dimension=384, metric='cosine',
                              spec=ServerlessSpec(cloud='aws', region='us-east-1'))
                time.sleep(10)
            self.index = pc.Index(config.PINECONE_INDEX)
            print(" Ready")
        except:
            print("  Pinecone unavailable")
            self.index = None
    
    def get_brand_context(self):
        return f"Brand: {self.config.BRAND_NAME}. Voice: {self.config.BRAND_VOICE}"

kb = KnowledgeBase(config)


  Pinecone...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Ready


In [14]:
class LLaMACaptionGenerator:
    def __init__(self, config, kb):
        self.config = config
        self.kb = kb
        self.llm = None

        if os.path.exists(config.LLAMA_MODEL_PATH):
            try:
                from llama_cpp import Llama
                print("\n LLaMA 2...")
                self.llm = Llama(
                    model_path=config.LLAMA_MODEL_PATH, n_ctx=2048,
                    n_gpu_layers=35 if config.DEVICE == "cuda" else 0,
                    verbose=False
                )
                print(" Ready")
            except:
                pass

    # ── public ────────────────────────────────────────────
    def generate(self, product):
        if self.llm:
            caption = self._llama_generate(product)
        else:
            caption = self._template_generate(product)
        hashtags = self._generate_hashtags(product)
        return f"{caption}\n\n{hashtags}"

    # ── strip meta-commentary ─────────────────────────────
    @staticmethod
    def _clean(raw):
        """
        Kills every pattern LLaMA actually outputs:

        Case A — multi-line, meta line on top:
            Sure! Here's a caption...
            💎 Shine bright...          ← we want this

        Case B — single line, meta + caption joined by colon:
            Sure, here's a caption for ShopSmart's Ring: 💎 Shine bright...
                                                         ^^^^^^^^^^^^^^^^^ we want this

        Case C — label prefix:
            Caption: 💎 Shine bright...

        Case D — already clean:
            💎 Shine bright...          ← pass through
        """
        text = raw.strip()
        if not text:
            return ""

        bad_starts = (
            "sure", "here", "caption:", "post:", "output:",
            "result:", "instagram", "below", "check", "i've",
            "i have", "let me", "for shopsmart", "for the",
            "ok", "great", "perfect"
        )

        # ── Case B / C: single-line with colon before the real caption ──
        # Find the LAST colon that is followed by an emoji or newline
        # e.g. "Sure, here's ... ShopSmart's Ring: 💎 ..."
        #                                          ^ split here
        if text.lower().startswith(tuple(bad_starts)):
            # walk through every colon; take the segment after the last one
            # that still has meaningful content
            parts = text.split(":")
            for i in range(len(parts) - 1, 0, -1):
                candidate = ":".join(parts[i:]).strip()
                if len(candidate) > 20:          # something real after it
                    text = candidate
                    break
            else:
                # no colon split worked → entire line is junk
                return ""

        # ── Case A: multi-line, drop bad leading lines ───────────────
        lines = [l.strip() for l in text.split("\n")]
        cleaned = []
        header_done = False
        for line in lines:
            if not header_done:
                if not line:
                    continue
                if line.lower().startswith(tuple(bad_starts)):
                    continue
                if line.lower().rstrip(":") in ("caption", "post", "output", "result"):
                    continue
                header_done = True
            cleaned.append(line)

        result = "\n".join(cleaned).strip()

        # ── final safety net ──────────────────────────────────────
        if result.lower().startswith(tuple(bad_starts)):
            return ""
        return result

    # ── LLaMA ─────────────────────────────────────────────
    def _llama_generate(self, product):
        prompt = (
            "[INST]\n"
            "###EXAMPLE1###\n"
            "Product: AirPods Pro 2nd Gen\n"
            "Price: $249\n"
            "Rating: 4.8/5 (3200 reviews)\n"
            "Description: Next-gen noise cancellation with adaptive transparency.\n"
            "Caption:\n"
            "🎧 Sound like never before.\n\n"
            "AirPods Pro 2 — adaptive noise cancellation\n"
            "that actually works. Loved by 3200+ buyers.\n\n"
            "⭐ 4.8/5 | 💰 $249\n\n"
            "👉 Link in bio — grab yours today!\n"
            "###END###\n\n"

            "###EXAMPLE2###\n"
            "Product: Nike Air Max 90\n"
            "Price: $129\n"
            "Rating: 4.5/5 (890 reviews)\n"
            "Description: Classic silhouette with visible Air cushioning.\n"
            "Caption:\n"
            "🔥 The classic is back.\n\n"
            "Nike Air Max 90 — iconic style meets\n"
            "serious comfort. 890+ people agree.\n\n"
            "⭐ 4.5/5 | 💰 $129\n\n"
            "🛒 Shop now — link in bio!\n"
            "###END###\n\n"

            f"###NOW###\n"
            f"Product: {product['title']}\n"
            f"Price: ${product['price']}\n"
            f"Rating: {product['rating']['rate']}/5 ({product['rating']['count']} reviews)\n"
            f"Description: {product['description'][:150]}\n"
            f"Caption:\n"
            "[/INST]"
        )

        try:
            out = self.llm(
                prompt, max_tokens=200, temperature=0.7,
                stop=["###", "[INST]", "Product:"]
            )
            raw = out["choices"][0]["text"].strip()
            caption = self._clean(raw)
            return caption if caption else self._template_generate(product)
        except:
            return self._template_generate(product)

    # ── templates (fallback) ──────────────────────────────
    def _template_generate(self, product):
        title  = product["title"]
        price  = product["price"]
        rating = product["rating"]["rate"]
        count  = product["rating"]["count"]
        desc   = product["description"][:100]

        templates = {
            "electronics": (
                f"⚡ {title}\n\n"
                f"{desc}\n\n"
                f"Loved by {count}+ buyers. ⭐ {rating}/5\n"
                f"💰 Only ${price}\n\n"
                f"🛒 Shop now — link in bio!"
            ),
            "jewelery": (
                f"✨ {title}\n\n"
                f"Timeless elegance. Stunning detail.\n"
                f"Loved by {count}+ customers. ⭐ {rating}/5\n"
                f"💎 Just ${price}\n\n"
                f"👉 Grab yours — link in bio!"
            ),
            "men's clothing": (
                f"🔥 {title}\n\n"
                f"Upgrade your wardrobe today.\n"
                f"{count}+ reviews. ⭐ {rating}/5\n"
                f"💰 Only ${price}\n\n"
                f"🛒 Shop now — link in bio!"
            ),
            "women's clothing": (
                f"👗 {title}\n\n"
                f"Style that turns heads.\n"
                f"{count}+ reviews. ⭐ {rating}/5\n"
                f"💰 Just ${price}\n\n"
                f"🛍️ Shop now — link in bio!"
            ),
        }
        return templates.get(product["category"], templates["electronics"])

    # ── hashtags ──────────────────────────────────────────
    def _generate_hashtags(self, product):
        base = {
            "electronics":       ["#Tech", "#Gadgets", "#Electronics", "#TechDeals"],
            "jewelery":          ["#Jewelry", "#JewelryLover", "#Accessories", "#GoldJewelry"],
            "men's clothing":   ["#MensFashion", "#MensStyle", "#OOTD", "#MensWear"],
            "women's clothing": ["#WomensFashion", "#WomensStyle", "#OOTD", "#Fashion"],
        }
        tags = list(self.config.BRAND_HASHTAGS)
        tags.extend(base.get(product["category"], ["#Shopping", "#OnlineShopping"]))
        tags.extend(["#Sale", "#ShopNow", "#NewArrival", "#Limited"])
        return " ".join(tags[:12])

caption_gen = LLaMACaptionGenerator(config, kb)
print(" Caption generator ready")


 Caption generator ready


In [15]:
class SDXLImageGenerator:
    def __init__(self, config):
        self.config = config
        self.pipe = None
        
        if config.DEVICE == "cuda":
            try:
                print("Loading SDXL...")
                self.pipe = StableDiffusionXLPipeline.from_pretrained(
                    config.SDXL_MODEL,
                    torch_dtype=torch.float16,
                    variant="fp16",
                    use_safetensors=True
                )
                self.pipe.to(config.DEVICE)
                
                try:
                    self.pipe.enable_xformers_memory_efficient_attention()
                except:
                    self.pipe.enable_attention_slicing(1)
                
                try:
                    self.pipe.enable_vae_slicing()
                except:
                    pass
                
                print("SDXL ready")
            except Exception as e:
                print(f"SDXL failed: {e}")
                self.pipe = None
    
    def create(self, product, pid):
        if self.pipe:
            try:
                return self._generate(product, pid)
            except Exception as e:
                print(f"Generation failed: {e}")
        return self._download_fallback(product, pid)
    
    def _generate(self, product, pid):
        print("Generating...")
        title = product["title"]
        category = product["category"]
        
        prompts = {
            "electronics": f"a {title} product on white background, studio lighting, professional product photo",
            "jewelery": f"a {title} on dark velvet, soft golden light, luxury product photo",
            "men's clothing": f"a {title} laid flat on white surface, studio lighting, professional photo",
            "women's clothing": f"a {title} laid flat on white surface, soft natural light, professional photo"
        }
        
        prompt = prompts.get(category, f"a {title} product on white background, studio lighting")
        
        negative = "person, people, human, face, hands, text, letters, watermark, logo, blurry"
        
        image = self.pipe(
            prompt=prompt,
            negative_prompt=negative,
            num_inference_steps=self.config.INFERENCE_STEPS,
            guidance_scale=self.config.GUIDANCE_SCALE,
            height=self.config.NATIVE_RES,
            width=self.config.NATIVE_RES
        ).images[0]
        
        image = image.resize((1080, 1080), Image.Resampling.LANCZOS)
        path = os.path.join(self.config.IMAGE_DIR, f"{pid}.png")
        image.save(path, "PNG", optimize=True)
        print(f"Done: {os.path.getsize(path)/1024:.1f}KB")
        return path
    
    def _download_fallback(self, product, pid):
        try:
            print("Downloading fallback image...")
            r = requests.get(product["image"], timeout=30, headers={"User-Agent": "Mozilla/5.0"})
            r.raise_for_status()
            img = Image.open(BytesIO(r.content)).convert("RGB")
            canvas = Image.new("RGB", (1080, 1080), (255, 255, 255))
            img.thumbnail((1080, 1080), Image.Resampling.LANCZOS)
            canvas.paste(img, ((1080 - img.width) // 2, (1080 - img.height) // 2))
            path = os.path.join(self.config.IMAGE_DIR, f"{pid}.jpg")
            canvas.save(path, "JPEG", quality=95)
            print(f"Fallback: {os.path.getsize(path)/1024:.1f}KB")
            return path
        except Exception as e:
            print(f"Failed: {e}")
            return None

image_gen = SDXLImageGenerator(config)

Loading SDXL...


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

vae_1_0/diffusion_pytorch_model.fp16.saf(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


SDXL ready


In [17]:
class BudgetManager:
    def __init__(self, config):
        self.config = config
        self.posts_this_month = 0
        self.spent = 0.0
    
    def can_post(self):
        if self.posts_this_month >= self.config.MONTHLY_POST_GOAL:
            return False, "Monthly goal reached"
        if self.spent + self.config.COST_PER_POST > self.config.MONTHLY_BUDGET:
            return False, "Budget exceeded"
        return True, "OK"
    
    def record_post(self):
        self.posts_this_month += 1
        self.spent += self.config.COST_PER_POST
    
    def get_status(self):
        return f"Posts: {self.posts_this_month}/{self.config.MONTHLY_POST_GOAL} | Spent: ${self.spent:.2f}/${self.config.MONTHLY_BUDGET}"

budget_manager = BudgetManager(config)
print(budget_manager.get_status())

Posts: 0/60 | Spent: $0.00/$500.0


In [18]:
cloudinary.config(
    cloud_name=config.CLOUDINARY_CLOUD_NAME,
    api_key=config.CLOUDINARY_API_KEY,
    api_secret=config.CLOUDINARY_API_SECRET
)
print("Cloudinary configured")

Cloudinary configured


In [19]:
class InstagramAPI:
    def __init__(self, config):
        self.config = config
        self.base = "https://graph.facebook.com/v24.0"
        self.posted = []
    
    def post(self, url, caption, product):
        try:
            print("Posting to Instagram...")
            r1 = requests.post(
                f"{self.base}/{self.config.INSTAGRAM_USER_ID}/media",
                params={
                    'image_url': url,
                    'caption': caption[:2200],
                    'access_token': self.config.PAGE_ACCESS_TOKEN
                }
            )
            
            if r1.status_code != 200:
                return self._simulate(url, caption, product)
            
            cid = r1.json()['id']
            print(f"Container: {cid}")
            time.sleep(10)
            
            r2 = requests.post(
                f"{self.base}/{self.config.INSTAGRAM_USER_ID}/media_publish",
                params={'creation_id': cid, 'access_token': self.config.PAGE_ACCESS_TOKEN}
            )
            
            if r2.status_code != 200:
                return self._simulate(url, caption, product)
            
            mid = r2.json()['id']
            post = {
                'id': mid,
                'status': 'published',
                'engagement': np.random.uniform(0.03, 0.06),
                'product': product['title'],
                'time': datetime.now().isoformat()
            }
            self.posted.append(post)
            return post
        except:
            return self._simulate(url, caption, product)
    
    def _simulate(self, url, caption, product):
        post = {
            'id': f"sim_{int(time.time())}",
            'status': 'simulated',
            'engagement': np.random.uniform(0.03, 0.06),
            'product': product['title'],
            'time': datetime.now().isoformat()
        }
        self.posted.append(post)
        return post

insta = InstagramAPI(config)
print("Instagram API ready")

Instagram API ready


In [20]:
print("Checking setup...")
print(f"Page ID: {config.FACEBOOK_PAGE_ID}")
print(f"IG User ID: {config.INSTAGRAM_USER_ID}")
print(f"Token: {config.PAGE_ACCESS_TOKEN[:20]}...")
print("\nReady to post!")

Checking setup...
Page ID: 966806373184726
IG User ID: 17841415236594174
Token: EAAUf7yytfZB0BQg1ang...

Ready to post!


In [21]:
class Tier2_SDXL:
    def __init__(self, products_df, caption_gen, image_gen, insta, budget_manager, config):
        self.products_df = products_df
        self.caption_gen = caption_gen
        self.image_gen = image_gen
        self.insta = insta
        self.budget_manager = budget_manager
        self.config = config
    
    def run(self, num_posts=3):
        print(f"\nTier 2: {num_posts} posts")
        print(budget_manager.get_status())
        print("="*60)
        
        success = 0
        
        for i in range(num_posts):
            print(f"\nPost {i+1}/{num_posts}")
            print("-"*60)
            
            can_post, msg = self.budget_manager.can_post()
            if not can_post:
                print(f"Stopped: {msg}")
                break
            
            product = self.products_df.sample(1).iloc[0].to_dict()
            print(f"Product: {product['title'][:50]}")
            print(f"Price: ${product['price']} | Rating: {product['rating']['rate']}⭐")
            
            caption = self.caption_gen.generate(product)
            print(f"\nCaption: {caption[:100]}...")
            
            pid = f"tier2_{int(time.time())}_{i}"
            img = self.image_gen.create(product, pid)
            if not img:
                continue
            
            try:
                result = cloudinary.uploader.upload(img, folder="instagram")
                url = result["secure_url"]
                print("Uploaded to Cloudinary")
            except Exception as e:
                print(f"Upload failed: {e}")
                continue
            
            post = self.insta.post(url, caption, product)
            if post:
                self.budget_manager.record_post()
                success += 1
                if post["status"] == "published":
                    print(f"✅ LIVE: https://www.instagram.com/p/{post['id']}/")
                else:
                    print(f"✅ Simulated ({post['engagement']:.2%})")
            
            if i < num_posts - 1:
                time.sleep(5)
        
        print("\n" + "="*60)
        print(f"Complete: {success}/{num_posts} posted")
        print(budget_manager.get_status())
        print("="*60)

tier2 = Tier2_SDXL(products_df, caption_gen, image_gen, insta, budget_manager, config)
print("Tier2 ready")

Tier2 ready


In [23]:
# Test single product
test_product = products_df.sample(1).iloc[0].to_dict()
print(f"Test product: {test_product['title']}")
test_caption = caption_gen.generate(test_product)
print(f"Caption:\n{test_caption}")

Test product: White Gold Plated Princess
Caption:
✨ White Gold Plated Princess

Timeless elegance. Stunning detail.
Loved by 400+ customers. ⭐ 3/5
💎 Just $9.99

👉 Grab yours — link in bio!

#ShopSmart #SmartShopping #Jewelry #JewelryLover #Accessories #GoldJewelry #Sale #ShopNow #NewArrival #Limited


In [24]:
tier2.run(num_posts=3)


Tier 2: 3 posts
Posts: 0/60 | Spent: $0.00/$500.0

Post 1/3
------------------------------------------------------------
Product: Rain Jacket Women Windbreaker Striped Climbing Rai
Price: $39.99 | Rating: 3.8⭐

Caption: 👗 Rain Jacket Women Windbreaker Striped Climbing Raincoats

Style that turns heads.
679+ reviews. ⭐ ...
Generating...


  0%|          | 0/30 [00:00<?, ?it/s]

Done: 2124.5KB
Uploaded to Cloudinary
Posting to Instagram...
Container: 18133602226507887
✅ LIVE: https://www.instagram.com/p/18056680739400857/

Post 2/3
------------------------------------------------------------
Product: Mens Casual Slim Fit
Price: $15.99 | Rating: 2.1⭐

Caption: 🔥 Mens Casual Slim Fit

Upgrade your wardrobe today.
430+ reviews. ⭐ 2.1/5
💰 Only $15.99

🛒 Shop now...
Generating...


  0%|          | 0/30 [00:00<?, ?it/s]

Done: 1374.8KB
Uploaded to Cloudinary
Posting to Instagram...
Container: 18133602349507887
✅ LIVE: https://www.instagram.com/p/18092639972286915/

Post 3/3
------------------------------------------------------------
Product: Silicon Power 256GB SSD 3D NAND A55 SLC Cache Perf
Price: $109.0 | Rating: 4.8⭐

Caption: ⚡ Silicon Power 256GB SSD 3D NAND A55 SLC Cache Performance Boost SATA III 2.5

3D NAND flash are ap...
Generating...


  0%|          | 0/30 [00:00<?, ?it/s]

Done: 1080.4KB
Uploaded to Cloudinary
Posting to Instagram...
Container: 18133602418507887
✅ LIVE: https://www.instagram.com/p/18098487298930503/

Complete: 3/3 posted
Posts: 3/60 | Spent: $15.00/$500.0
